# Aula 4 - Manipulação de df: groupby e merge


### Objetivos

Apresentar como unir dataframes e realizar cálculos com dados agrupados

____________________________

### Habilidades a serem desenvolvidas nessa aula

Ao final da aula o aluno deve:

- Saber como concatenar dataframes,
- Conseguir agrupar os dados e aplicar vários métodos à eles


____
____
____

## Titanic

O arquivo que usaremos hoje é relativo ao Titanic! Essa é uma das bases mais famosas de ciência de dados. Você pode saber mais sobre estes dados [clicando aqui!](https://www.kaggle.com/c/titanic)

In [1]:
import pandas as pd
import numpy as np

In [2]:
# lê dataframe do arquivo titanic.csv 
df = pd.read_csv("data/titanic.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


E se quisessemos calcular a média de Fare por Pclasse utilizando apenas o que aprendemos até agora?

Ou de forma mais automática:

In [3]:
for i in df.Pclass.sort_values().unique():
    print(f'Ticket médio da classe {i}: ', df[df['Pclass']==i].Fare.mean())

Ticket médio da classe 1:  84.15468749999992
Ticket médio da classe 2:  20.66218315217391
Ticket médio da classe 3:  13.675550101832997


E se quisessemos calcular a média por Pclass e Sex?

### Groupby
Assim como no SQL, no pandas também temos um método com o qual podemos agregar os dados. O `groupby` primeiro separa nossos dados em grupos definidos dentro do método,  aplica um tipo de operação usando agregação, transformação, filtragem ou até uma função própria e, por fim, junta os resultados encontrados.
<br>

<img src="groupby.png"  style="width: 700px" >

Exemplo de aplicação da função de agregação `mean`
<br><br><br>

Utilizar o `groupby` é o mesmo que fazer a sequência:

   1. Dividir os dados em grupos utilizando um critério
    
   2. Aplicar uma função em cada um dos grupos separadamente
    
   3. Combinar o resultado em uma estrutura de dados

#### Funções de agregação
Com essas funções podemos aplicar operações estatísticas nos nossos dados. Exemplos:<br>
`mean`, `std`, `max`, `min`, `count`, `sum`, `var`. <br>
Quando queremos aplicar apenas uma dessas operações podemos chamá-las diretamente após o `groupby`:


In [4]:
# Agrupa por Pclass e Sex e calcula a média de cada grupo


Aqui agregamos os dados por Pclass e Sex e em todas as colunas numéricas foi calculada a média. Se quiséssemos a média de apenas uma coluna poderíamos adicioná-la ao final da nossa sentença:

In [5]:
# Queremos apenas a média de idade considerando a classe e o sexo
df.groupby(["Pclass", "Sex"]).mean()[['Age']]

Age
Pclass Sex              
1      female  34.611765
       male    41.281386
2      female  28.722973
       male    30.740707
3      female  21.750000
       male    26.507589

Ou de modo mais eficiente:

In [6]:
df.groupby(["Pclass", "Sex"])[['Age']].mean()

Age
Pclass Sex              
1      female  34.611765
       male    41.281386
2      female  28.722973
       male    30.740707
3      female  21.750000
       male    26.507589

Note que `df.groupby('A').colname.mean()` é mais eficiente que `df.groupby('A').mean().colname` pois a agregação só será realizada na coluna de interesse (colname).

Quando queremos aplicar mais de uma operação chamamos o método `.agg()`

In [7]:
df.groupby(["Pclass"]).agg(['mean','max','min'])

PassengerId           Survived                Age              \
              mean  max min      mean max min       mean   max   min   
Pclass                                                                 
1       461.597222  890   2  0.629630   1   0  38.233441  80.0  0.92   
2       445.956522  887  10  0.472826   1   0  29.877630  70.0  0.67   
3       439.154786  891   1  0.242363   1   0  25.140620  74.0  0.42   

           SibSp             Parch               Fare                 
            mean max min      mean max min       mean       max  min  
Pclass                                                                
1       0.416667   3   0  0.356481   4   0  84.154687  512.3292  0.0  
2       0.402174   3   0  0.380435   3   0  20.662183   73.5000  0.0  
3       0.615071   8   0  0.393075   6   0  13.675550   69.5500  0.0

Para operações distintas em colunas distintas passamos um dicionário com o nome da coluna como chave e a operação como valor

In [8]:
import numpy as np
df.groupby(['Pclass']).agg({'Embarked': pd.Series.mode, 'Fare': np.mean})

,Embarked,Fare
Pclass,,
1,S,84.154687
2,S,20.662183
3,S,13.675550


Reparem que a coluna utilizada no `groupby` virou um index do nosso df. Para convertê-la em coluna novamente temos duas formas: <br>
  1. chamar o parâmetro `as_index=False` dentro do `groupby`
  2. aplicar `.reset_index()` ao final da sentença

In [9]:
# exemplo com as_index = False


In [10]:
# exemplo com .reset_index()


_____________
_____________
**Exercício:** Existe diferença de sobrevivência por portão de embarque? E diferença no preço do ticket? Porque você acha que tem essa diferença?

In [11]:
# Resposta
df.groupby(['Embarked','Pclass'])[['Survived','Fare']].agg({'mean'})



Survived        Fare
                     mean        mean
Embarked Pclass                      
C        1       0.694118  104.718529
         2       0.529412   25.358335
         3       0.378788   11.214083
Q        1       0.500000   90.000000
         2       0.666667   12.350000
         3       0.375000   11.183393
S        1       0.582677   70.364862
         2       0.463415   20.327439
         3       0.189802   14.644083

______________
_____________

E se quiséssemos criar uma coluna nova que contenham o valor médio do Fare por Pclass?

### Criando coluna com dado agregado

Queremos que todas as pessoas da primeira classe tenham o valor 84.15 nessa nova coluna, todas da segunda classe tenham o valor 20.66 e da terceira classe 13.67. <br>
Podemos tentar:

In [12]:
df.groupby('Pclass')[["Fare"]].mean()

,Fare
Pclass,
1,84.154687
2,20.662183
3,13.675550


In [13]:
df["Fare_Mean"] = df.groupby('Pclass')["Fare"].mean()

df.head(7)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,84.154687
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,20.662183
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,13.675550
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,NaN
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,NaN
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,NaN


Xiiii... deu ruim...
<br>
<br>


#### Transformação dos dados
Ao aplicarmos o método `.transform()` temos como retorno um objeto com o mesmo index do df de origem contendo a a transformação realizada para cada uma das linhas. Dessa forma podemos utilizar esse método e apenas criar uma coluna nova no nosso df.
<br>

Ele será muito **útil na criação de novas features** para os modelos.

In [14]:
df.groupby('Pclass')[["Fare"]].transform('mean')

,Fare
0,13.675550
1,84.154687
2,13.675550
3,84.154687
4,13.675550
...,...
886,20.662183
887,84.154687
888,13.675550
889,84.154687


In [15]:
df["Fare_Mean"] = df.groupby('Pclass')["Fare"].transform('mean')
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,13.675550
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,84.154687
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,13.675550
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,84.154687
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,13.675550
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,13.675550
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,84.154687
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,13.675550
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,13.675550
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,20.662183


Podemos aplicar tanto as operações mencionadas na agregação quanto uma função `lambda`:

In [16]:
df['variacao_max_min'] = df.groupby('Pclass')[["Fare"]].transform(lambda x: x.max() - x.min())
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean,variacao_max_min
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,13.675550,69.5500
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,84.154687,512.3292
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,13.675550,69.5500
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,84.154687,512.3292
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,13.675550,69.5500
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,13.675550,69.5500
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,84.154687,512.3292
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,13.675550,69.5500
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,13.675550,69.5500
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,20.662183,73.5000


Ou até mesmo passar funções construídas:

In [17]:
def funcao_max_menos_min(x):
    return x.max() - x.min()

In [18]:
df.groupby('Pclass')[["Fare"]].transform(funcao_max_menos_min)

,Fare
0,69.5500
1,512.3292
2,69.5500
3,512.3292
4,69.5500
...,...
886,73.5000
887,512.3292
888,69.5500
889,512.3292


Também podemos preencher os valores nulos com a média de cada grupo

In [19]:
# verificando quantidade de nulos por coluna
df.isna().sum()

PassengerId           0
Survived              0
Pclass                0
Name                  0
Sex                   0
Age                 177
SibSp                 0
Parch                 0
Ticket                0
Fare                  0
Cabin               687
Embarked              2
Fare_Mean             0
variacao_max_min      0
dtype: int64

Para preencher os nulos utilizaremos o método `.fillna()` que vimos em aula:

In [20]:
df['Age_sem_nulo'] = df.groupby(['Pclass'])[['Age']].transform(lambda x: x.fillna(x.mean()))

In [21]:
df.isna().sum()

PassengerId           0
Survived              0
Pclass                0
Name                  0
Sex                   0
Age                 177
SibSp                 0
Parch                 0
Ticket                0
Fare                  0
Cabin               687
Embarked              2
Fare_Mean             0
variacao_max_min      0
Age_sem_nulo          0
dtype: int64

In [22]:
# Conferindo o preenchimento de nulos
# idade média por Pclass
df.groupby(['Pclass'])[['Age']].mean()

,Age
Pclass,
1,38.233441
2,29.877630
3,25.140620


In [23]:
# selecionando a parte do df que tem idade nula
df[df.Age.isna()].head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean,variacao_max_min,Age_sem_nulo
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,13.675550,69.5500,25.140620
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S,20.662183,73.5000,29.877630
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C,13.675550,69.5500,25.140620
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C,13.675550,69.5500,25.140620
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q,13.675550,69.5500,25.140620
29,30,0,3,"Todoroff, Mr. Lalio",male,NaN,0,0,349216,7.8958,NaN,S,13.675550,69.5500,25.140620
31,32,1,1,"Spencer, Mrs. William Augustus (Marie Eugenie)",female,NaN,1,0,PC 17569,146.5208,B78,C,84.154687,512.3292,38.233441
32,33,1,3,"Glynn, Miss. Mary Agatha",female,NaN,0,0,335677,7.7500,NaN,Q,13.675550,69.5500,25.140620
36,37,1,3,"Mamee, Mr. Hanna",male,NaN,0,0,2677,7.2292,NaN,C,13.675550,69.5500,25.140620
42,43,0,3,"Kraeff, Mr. Theodor",male,NaN,0,0,349253,7.8958,NaN,C,13.675550,69.5500,25.140620


_________________________
_________________________
**Exercício:** Crie uma coluna com a média de Fare e outra com a média de idade para cada classe da coluna Survived. Você consegue fazer isso de uma única vez?

In [24]:
df.groupby(['Survived'])[['Fare','Age']].transform('mean')

,Fare,Age
0,22.117887,30.626179
1,48.395408,28.343690
2,48.395408,28.343690
3,48.395408,28.343690
4,22.117887,30.626179
...,...,...
886,22.117887,30.626179
887,48.395408,28.343690
888,22.117887,30.626179
889,48.395408,28.343690


In [25]:
# Resposta
df[['Mean_Fare','Mean_Age']] = df.groupby('Survived')[['Fare','Age']].transform('mean')

df.head(15)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean,variacao_max_min,Age_sem_nulo,Mean_Fare,Mean_Age
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,13.675550,69.5500,22.00000,22.117887,30.626179
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,84.154687,512.3292,38.00000,48.395408,28.343690
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,13.675550,69.5500,26.00000,48.395408,28.343690
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,84.154687,512.3292,35.00000,48.395408,28.343690
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,13.675550,69.5500,35.00000,22.117887,30.626179
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,13.675550,69.5500,25.14062,22.117887,30.626179
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,84.154687,512.3292,54.00000,22.117887,30.626179
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,13.675550,69.5500,2.00000,22.117887,30.626179
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,13.675550,69.5500,27.00000,48.395408,28.343690
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,20.662183,73.5000,14.00000,48.395408,28.343690


_________________________
_________________________

## Cruzamento e concatenação de bases

Também é possível fazer **cruzamento de bases** com o pandas. 

Pra quem conhece SQL: esses são os joins!

Pra quem conhece Excel: essa é uma forma de fazer o procv!

Vamos supor que temos as notas de duas provas dos alunos separas em sheets diferentes do excel e queremos juntar essa notas em um único df.

In [26]:
# ler os dados de diferentes sheets do mesmo excel "notas.xlsx"
df1 = pd.read_excel("notas.xlsx", sheet_name="notas1")
df2 = pd.read_excel("notas.xlsx", sheet_name="notas2")

In [27]:
df1

,RA,aluno,prova1
0,1,joão,10
1,4,leia,10
2,2,maria,9
3,3,han,8
4,5,luke,7
5,7,obi wan,10


In [28]:
df2

,RA,aluno,prova2
0,1,joão,10
1,4,leia,10
2,2,maria,9
3,3,han,8
4,5,luke,7
5,6,anakin,10


Repare que temos alunos distintos nos dois df

Diferentes tipos de join

<img src="join_exemplo2.png" />
Fonte: https://towardsdatascience.com/python-pandas-dataframe-join-merge-and-concatenate-84985c29ef78

O pandas possui dois métodos específicos para trabalharmos com o join de colunas entre df: `.merge()` e `.join()`. O `.merge()` fornece mais flexibilidade de trabalho e iremos utilizar e ele.

### pd.merge()
pd.merge(
    left,
    right,
    how="inner",
    on=None,
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y")
)

In [29]:
df1.merge(df2, how="outer", on="RA")

,RA,aluno_x,prova1,aluno_y,prova2
0,1,joão,10.0,joão,10.0
1,4,leia,10.0,leia,10.0
2,2,maria,9.0,maria,9.0
3,3,han,8.0,han,8.0
4,5,luke,7.0,luke,7.0
5,7,obi wan,10.0,NaN,NaN
6,6,NaN,NaN,anakin,10.0


In [30]:
df1.merge(df2, how="outer", on=["RA", "aluno"])

,RA,aluno,prova1,prova2
0,1,joão,10.0,10.0
1,4,leia,10.0,10.0
2,2,maria,9.0,9.0
3,3,han,8.0,8.0
4,5,luke,7.0,7.0
5,7,obi wan,10.0,NaN
6,6,anakin,NaN,10.0


### pd.concat()
Diferente do `.merge()` e `.join()` que operam apenas com colunas, com o `.concat()` podemos especificar se queremos **concatenar em linhas ou colunas**.
Na concatenação de colunas o `.concat()` somente considera o index dos df e, por isso, não podemos especificar colunas como feito com o `.merge()`.

`pd.concat(
    objs,
    axis=0,
    join="outer",
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=True,
)`


In [31]:
pd.concat([df1, df2], axis=1, join="inner")

,RA,aluno,prova1,RA,aluno,prova2
0,1,joão,10,1,joão,10
1,4,leia,10,4,leia,10
2,2,maria,9,2,maria,9
3,3,han,8,3,han,8
4,5,luke,7,5,luke,7
5,7,obi wan,10,6,anakin,10


Repare que ao concatenar diretamente pelo index ele juntou o aluno obi wan com o anakin. 

Ao concatenar dois df nas linhas, o `.concat()` irá considerar o nome das colunas. Se temos colunas com nomes distintos e utilizamos o parâmetro join='inner', ele irá ignorar essas colunas: 

In [32]:
pd.concat([df1, df2], axis=0, join="inner")

,RA,aluno
0,1,joão
1,4,leia
2,2,maria
3,3,han
4,5,luke
5,7,obi wan
0,1,joão
1,4,leia
2,2,maria
3,3,han


Para que ele considere todas as colunas utilizamos o argumento 
```python 
join="outer" 
```

In [33]:
pd.concat([df1, df2], join="outer")

,RA,aluno,prova1,prova2
0,1,joão,10.0,NaN
1,4,leia,10.0,NaN
2,2,maria,9.0,NaN
3,3,han,8.0,NaN
4,5,luke,7.0,NaN
5,7,obi wan,10.0,NaN
0,1,joão,NaN,10.0
1,4,leia,NaN,10.0
2,2,maria,NaN,9.0
3,3,han,NaN,8.0


## Exercícios

1. Considere a existência de três tabelas distintas:
* customer.csv : Possui a informação dos clientes em duas colunas: customer id  customer name
* products.csv : Conté informação dos produtos vendidos pela empresa em três colunas - p_id (product id), product (name) e price
* sales.csv : Contém informações das vendas realizadas em seis colunas - sale_id, c_id (customer id), p_id (product_id), qty (quantity sold), store (name)

Conhecendo as bases e utilizando os métodos de concatenação de bases responda:


In [34]:
clientes = pd.read_csv('./data/customer.csv')
produtos = pd.read_csv('./data/products.csv')
vendas = pd.read_csv('./data/sales.csv')
produtos


,p_id,product,price
0,1,Hard Disk,80
1,2,RAM,90
2,3,Monitor,75
3,4,CPU,55
4,5,Keyboard,20
5,6,Mouse,10
6,7,Motherboard,50
7,8,Power supply,20


a) Quais produtos não foram vendidos?

In [35]:
nao_vendidos = produtos.merge(vendas, how='outer', on='product')


b) Quantos clientes não realizaram uma compra? 

In [36]:
clientes_nao_compraram = clientes.merge(vendas, how='outer', on='c_id')

clientes_nao_compraram[clientes_nao_compraram['sale_id'].isna()]['Customer']

9     King
10    Ronn
11     Jem
12     Tom
Name: Customer, dtype: object

c) Liste a quantidade vendida e o faturamento de cada produto 

In [37]:
venda_fatura = produtos.merge(vendas, how='inner', on='product')



venda_fatura['total_earnings'] = venda_fatura['qty'] * venda_fatura['price']



venda_fatura.groupby('product')[['qty','total_earnings']].sum().reset_index()






,product,qty,total_earnings
0,CPU,1,55
1,Monitor,12,900
2,RAM,7,630


d) Liste a quantidade vendida de cada produto por loja

In [38]:
venda_fatura.groupby(['store','product'])[['qty']].sum().reset_index()



,store,product,qty
0,ABC,Monitor,10
1,ABC,RAM,3
2,DEF,CPU,1
3,DEF,Monitor,2
4,DEF,RAM,4


e) Qual loja teve maior faturamento?

In [39]:
venda_fatura.groupby(['store'])[['total_earnings']].sum().reset_index()

,store,total_earnings
0,ABC,1020
1,DEF,565


f) Qual produto foi o mais vendido?

In [66]:
venda_fatura.groupby(['product'])[['qty']].sum().sort_values('qty',ascending=False).head(1).reset_index()

,product,qty
0,Monitor,12


## Referências
https://pandas.pydata.org/docs/user_guide/groupby.html <br>
https://pandas.pydata.org/docs/user_guide/merging.html <br> 
https://towardsdatascience.com/python-pandas-dataframe-join-merge-and-concatenate-84985c29ef78 <br>
[When to use pandas transform function](https://towardsdatascience.com/when-to-use-pandas-transform-function-df8861aa0dcf)

## Material extra

### Outros parâmetros do groupby por default
* as_index
* sort
* dropna # exclui nans nas keys

<br> Em todas o default do python é True <br>
df.groupby('Pclass', sort=False)["Fare"].mean()

In [41]:
# dropna
df_list = [[1, 2, 3], [1, None, 4], [2, 1, 3], [1, 2, 2]]
df_dropna = pd.DataFrame(df_list, columns=["a", "b", "c"])
df_dropna

,a,b,c
0,1,2.0,3
1,1,NaN,4
2,2,1.0,3
3,1,2.0,2


In [42]:
# Default ``dropna`` is set to True, which will exclude NaNs in keys
df_dropna.groupby(by=["b"], dropna=True).sum()

,a,c
b,,
1.0,2,3
2.0,2,5


In [43]:
df_dropna.groupby(by=["b"], dropna=False).sum()

,a,c
b,,
1.0,2,3
2.0,2,5
NaN,1,4


Repare que podemos chamar qualquer função do `pd.Series` ou  do `numpy`

In [44]:
df.groupby(["Survived"]).mean()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Fare_Mean,variacao_max_min,Age_sem_nulo,Mean_Fare,Mean_Age
Survived,,,,,,,,,,,
0,447.016393,2.531876,30.626179,0.553734,0.329690,22.117887,25.180166,134.769464,29.819165,22.117887,30.626179
1,444.368421,1.950292,28.343690,0.473684,0.464912,48.395408,43.479643,246.630764,28.448040,48.395408,28.343690


### Função Lambda
Uma função lambda nada mais é que uma **forma alternativa de declarar uma função**, de um jeito mais direto

In [45]:
# função que retorna o dobro de um número usando def
def dobro(x):
    
    return 2*x

dobro(2)

4

In [46]:
# função que retorna o dobro de um número usando lambda x
faz_dobro = lambda x: 2*x

In [47]:
faz_dobro(6)

12

### Apply
O método `.apply()` recebe uma função como input e aplica ela para todo o df como se fosse um loop. Se você quiser que essa função seja aplicada ao longo das colunas deve considerar axis=0 e ao longo das linhas axis=1)

In [48]:
df.groupby(['Pclass']).apply(lambda x: x.describe())

PassengerId    Survived  Pclass         Age       SibSp  \
Pclass                                                                  
1      count   216.000000  216.000000   216.0  186.000000  216.000000   
       mean    461.597222    0.629630     1.0   38.233441    0.416667   
       std     246.737616    0.484026     0.0   14.802856    0.611898   
       min       2.000000    0.000000     1.0    0.920000    0.000000   
       25%     270.750000    0.000000     1.0   27.000000    0.000000   
       50%     472.000000    1.000000     1.0   37.000000    0.000000   
       75%     670.500000    1.000000     1.0   49.000000    1.000000   
       max     890.000000    1.000000     1.0   80.000000    3.000000   
2      count   184.000000  184.000000   184.0  173.000000  184.000000   
       mean    445.956522    0.472826     2.0   29.877630    0.402174   
       std     250.852161    0.500623     0.0   14.001077    0.601633   
       min      10.000000    0.000000     2.0    0.670000    0.000000   
       25%     234.500000    0.000000     2.0   23.000000    0.000000   
       50%     435.500000    0.000000     2.0   29.000000    0.000000   
       75%     668.000000    1.000000     2.0   36.000000    1.000000   
       max     887.000000    1.000000     2.0   70.000000    3.000000   
3      count   491.000000  491.000000   491.0  355.000000  491.000000   
       mean    439.154786    0.242363     3.0   25.140620    0.615071   
       std     264.441453    0.428949     0.0   12.495398    1.374883   
       min       1.000000    0.000000     3.0    0.420000    0.000000   
       25%     200.000000    0.000000     3.0   18.000000    0.000000   
       50%     432.000000    0.000000     3.0   24.000000    0.000000   
       75%     666.500000    0.000000     3.0   32.000000    1.000000   
       max     891.000000    1.000000     3.0   74.000000    8.000000   

                   Parch        Fare     Fare_Mean  variacao_max_min  \
Pclass                                                                 
1      count  216.000000  216.000000  2.160000e+02      2.160000e+02   
       mean     0.356481   84.154687  8.415469e+01      5.123292e+02   
       std      0.693997   78.380373  1.566825e-13      2.962724e-12   
       min      0.000000    0.000000  8.415469e+01      5.123292e+02   
       25%      0.000000   30.923950  8.415469e+01      5.123292e+02   
       50%      0.000000   60.287500  8.415469e+01      5.123292e+02   
       75%      0.000000   93.500000  8.415469e+01      5.123292e+02   
       max      4.000000  512.329200  8.415469e+01      5.123292e+02   
2      count  184.000000  184.000000  1.840000e+02      1.840000e+02   
       mean     0.380435   20.662183  2.066218e+01      7.350000e+01   
       std      0.690963   13.417399  4.631130e-14      0.000000e+00   
       min      0.000000    0.000000  2.066218e+01      7.350000e+01   
       25%      0.000000   13.000000  2.066218e+01      7.350000e+01   
       50%      0.000000   14.250000  2.066218e+01      7.350000e+01   
       75%      1.000000   26.000000  2.066218e+01      7.350000e+01   
       max      3.000000   73.500000  2.066218e+01      7.350000e+01   
3      count  491.000000  491.000000  4.910000e+02      4.910000e+02   
       mean     0.393075   13.675550  1.367555e+01      6.955000e+01   
       std      0.888861   11.778142  1.031338e-13      3.698591e-13   
       min      0.000000    0.000000  1.367555e+01      6.955000e+01   
       25%      0.000000    7.750000  1.367555e+01      6.955000e+01   
       50%      0.000000    8.050000  1.367555e+01      6.955000e+01   
       75%      0.000000   15.500000  1.367555e+01      6.955000e+01   
       max      6.000000   69.550000  1.367555e+01      6.955000e+01   

              Age_sem_nulo   Mean_Fare    Mean_Age  
Pclass                                              
1      count    216.000000  216.000000  216.000000  
       mean      38.233441   38.662993   29.189056  
       std       13.731315   1

Uma grande funcionalidade do pandas é que com o método `apply()` podemos aplicar uma **função** (muitas vezes, uma **função lambda**) a uma coluna ou linha de um DataFrame



Vamos selecionar a coluna de idades...

In [49]:
df["Age"]

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    27.0
887    19.0
888     NaN
889    26.0
890    32.0
Name: Age, Length: 891, dtype: float64

Aplicando uma função lambda **a todos os elementos da coluna**, ou seja, **à todas as linhas da tabela, daquela coluna específica**:

Tomando cada idade + 2, usando a função lambda definida.

Essa função lambda é equivalente a:

```python

def funcao(x):

    return x + 2
```

In [50]:
df["Age"].apply(lambda x: x + 2)

0      24.0
1      40.0
2      28.0
3      37.0
4      37.0
       ... 
886    29.0
887    21.0
888     NaN
889    28.0
890    34.0
Name: Age, Length: 891, dtype: float64

In [51]:
def funcao(x):
    return x + 2

df.Age.apply(funcao)

0      24.0
1      40.0
2      28.0
3      37.0
4      37.0
       ... 
886    29.0
887    21.0
888     NaN
889    28.0
890    34.0
Name: Age, Length: 891, dtype: float64

In [52]:
df.Age.transform(funcao)

0      24.0
1      40.0
2      28.0
3      37.0
4      37.0
       ... 
886    29.0
887    21.0
888     NaN
889    28.0
890    34.0
Name: Age, Length: 891, dtype: float64

Um outro exemplo:

In [53]:
# função: transforma todos os números em string, e concatena "!!!!!!!!!" à string
df["Age"].apply(lambda x: str(x) + "!!!!!!!!!")

0      22.0!!!!!!!!!
1      38.0!!!!!!!!!
2      26.0!!!!!!!!!
3      35.0!!!!!!!!!
4      35.0!!!!!!!!!
           ...      
886    27.0!!!!!!!!!
887    19.0!!!!!!!!!
888     nan!!!!!!!!!
889    26.0!!!!!!!!!
890    32.0!!!!!!!!!
Name: Age, Length: 891, dtype: object

Vamos usar uma função lambda para **extrair o sobrenome** dos nomes dos passageiros

Pra extrarir o sobrenome, note que este está separada do resto do nome por vírgula.

Para perceber isso, dê uma olhada na coluna de nomes:

In [54]:
df["Name"]

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
886                                Montvila, Rev. Juozas
887                         Graham, Miss. Margaret Edith
888             Johnston, Miss. Catherine Helen "Carrie"
889                                Behr, Mr. Karl Howell
890                                  Dooley, Mr. Patrick
Name: Name, Length: 891, dtype: object

Portanto, podemos usar a função para strings `split(",")`, com quebra na vírgula, e depois selecionar o primeiro elemento da lista gerada!

Vamos aproveitar e **criar uma nova coluna da base**, com os sobrenomes!

In [55]:
df["Surname"] = df["Name"].apply(lambda x: x.split(",")[0])

In [56]:
df["Surname"]

0         Braund
1        Cumings
2      Heikkinen
3       Futrelle
4          Allen
         ...    
886     Montvila
887       Graham
888     Johnston
889         Behr
890       Dooley
Name: Surname, Length: 891, dtype: object

### Apply com funções

E se quisessemos comparar o quanto cada passageiro pagou a mais ou a menos da média do Fare?

In [57]:
def f(group):
    return pd.DataFrame({'Fare_original': group,
                         'Fare_variacao': group - group.mean()})

df[['Fare_original','Fare_variacao']] = df.groupby('Pclass')['Fare'].apply(f)

In [58]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean,variacao_max_min,Age_sem_nulo,Mean_Fare,Mean_Age,Surname,Fare_original,Fare_variacao
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,13.675550,69.5500,22.0,22.117887,30.626179,Braund,7.2500,-6.425550
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,84.154687,512.3292,38.0,48.395408,28.343690,Cumings,71.2833,-12.871387
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,13.675550,69.5500,26.0,48.395408,28.343690,Heikkinen,7.9250,-5.750550
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,84.154687,512.3292,35.0,48.395408,28.343690,Futrelle,53.1000,-31.054687
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,13.675550,69.5500,35.0,22.117887,30.626179,Allen,8.0500,-5.625550


#### Transform X Apply
Com uma função de agregação o `.transform()` retorna um df que tem a mesma quantidade de linhas que o df original enquanto o `.apply` retorna o agregado por grupos.

### Filtros
O filtro retorna apenas um subset do nosso df. Aqui podemos aplicar filtros mais elaborados do que os vistos na última aula. <br>
Podemos, por exemplo, eliminar categorias do df que possuem apenas alguns elementos:

In [59]:
df.SibSp.value_counts()

0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: SibSp, dtype: int64

In [60]:
df.shape

(891, 20)

In [61]:
def filter_func(x):
    return x['Fare'] - x.Fare_Mean < 100

# df_filter = df.groupby(['SibSp']).filter(lambda x: filter_func(x))

df_filter = df.groupby(['SibSp']).filter(lambda x: len(x) >20)
df_filter.shape

(845, 20)

In [62]:
df_filter.SibSp.value_counts()

0    608
1    209
2     28
Name: SibSp, dtype: int64

Vamos supor que antes de afundar o titanic, o time de hapiness quisesse promover uma jogatina para os grupos (segmentado por classe e sexo) que possuem idade média acima de 30 anos.

In [63]:
df.groupby(['Pclass','Sex'])[['Age']].mean()

Age
Pclass Sex              
1      female  34.611765
       male    41.281386
2      female  28.722973
       male    30.740707
3      female  21.750000
       male    26.507589

como podemos filtrar nosso df para termos apenas os passageiros que pertecem a essas segmentações escolhidas?

In [64]:
df.groupby(['Pclass','Sex']).filter(lambda x: x['Age'].mean()>30)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean,variacao_max_min,Age_sem_nulo,Mean_Fare,Mean_Age,Surname,Fare_original,Fare_variacao
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,84.154687,512.3292,38.00000,48.395408,28.343690,Cumings,71.2833,-12.871387
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,84.154687,512.3292,35.00000,48.395408,28.343690,Futrelle,53.1000,-31.054687
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,84.154687,512.3292,54.00000,22.117887,30.626179,McCarthy,51.8625,-32.292187
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S,84.154687,512.3292,58.00000,48.395408,28.343690,Bonnell,26.5500,-57.604687
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S,20.662183,73.5000,29.87763,48.395408,28.343690,Williams,13.0000,-7.662183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C,84.154687,512.3292,56.00000,48.395408,28.343690,Potter,83.1583,-0.996387
883,884,0,2,"Banfield, Mr. Frederick James",male,28.0,0,0,C.A./SOTON 34068,10.5000,NaN,S,20.662183,73.5000,28.00000,22.117887,30.626179,Banfield,10.5000,-10.162183
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,20.662183,73.5000,27.00000,22.117887,30.626179,Montvila,13.0000,-7.662183
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,84.154687,512.3292,19.00000,48.395408,28.343690,Graham,30.0000,-54.154687
